# Hyperparameter search over CNN model based on 5-fold cross-validation

User c is completely removed as this is the test set.

The remaining users A,B,D,E and F make up each fold.

For example, fold 1 will train on users B,D,E and F then evaluate on A (and so on for each fold).

Choice of model justified from "7_CNN_model_comparison.ipynb"

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [0]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\baseline_model':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/doppler_spectrograms/3/'

RESULTS_PATH = BASE_PATH + 'results/CNN_hyperparameter_search/'
if not os.path.exists(RESULTS_PATH):
    os.makedirs(RESULTS_PATH)
    
# REPORT_PATH = BASE_PATH + 'reports/baseline_model/leave_one_user_out_cnn_64_128/'
    
from src.models.original_models import cnn_64_128
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


Import remaining packages

In [0]:
import numpy as np
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D

In [0]:
! pip install scikit-optimize
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.callbacks import CheckpointSaver

### Check this cell

In [0]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

## Experiment Setup

In [0]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
nb_classes = len(target_names)
batch_size = 100
nb_epoch = 20

# input image dimensions
img_rows, img_cols = 75, 75
# user c excluded as this is reserved for final evaluation
users = ["a", "b", "d", "e", "f"] 

In [0]:
def load_data(user_letter):
    with open(DATA_PATH + user_letter + "_data.pkl", 'rb') as data_file:
        data = pickle.load(data_file)
        data = data.reshape(data.shape[0], 1, 75, 75)
        
    with open(DATA_PATH + user_letter + "_labels.pkl", 'rb') as labels_file:
        labels = pickle.load(labels_file)
        labels = np.reshape(labels, (len(labels), 1))

        
    return data, labels

In [0]:
datasets = {}
for user in users:
    data, labels = load_data(user)
    datasets[user] = {"data":data, "labels":labels}

In [0]:
def split_train_validation(validation_user):
    train_data = np.empty((1,1,75,75))
    train_labels = np.empty((1,1))
    validation_data = []
    validation_labels = []
    for user in users:
        data = datasets[user]["data"]
        labels = datasets[user]["labels"]
        if user == validation_user:
            validation_data = data
            validation_labels = labels
            
        else:
            train_data = np.concatenate((train_data, data))
            train_labels = np.concatenate((train_labels, labels))
            
    train_labels = np_utils.to_categorical(train_labels, nb_classes)
    validation_labels = np_utils.to_categorical(validation_labels, nb_classes)
    train_data = train_data.astype('float32')
    validation_data = validation_data.astype('float32')
    train_data /= 255
    validation_data /= 255 
    
    return {
        "train_data": train_data,
        "train_labels": train_labels,
        "validation_data": validation_data,
        "validation_labels": validation_labels
       }

## Define Model

In [0]:
def make_model(nb_filters, img_rows, img_cols, nb_classes):
    model = Sequential(name=nb_filters)
    nb_filters = nb_filters.split("-")
    size_1 = int(nb_filters[0])
    size_2 = int(nb_filters[1])

    model.add(Convolution2D(size_1, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
    model.add(Convolution2D(size_1, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(size_2, (3, 3), padding='same', activation='relu'))
    model.add(Convolution2D(size_2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

## Variables to save results

In [0]:
results = {}
for model in models:
    results[model.name] = {}
    for user in users:
        results[model.name][user] = {}

## Hyperparameter Search

### Fixed Parameters

### Parameters to Search

### Objective Function to Minimize

In [0]:
@use_named_args(space)
def objective(**params):
    average_accuracy = 0
    average_loss = 0
    for user in users:
        data_split = split_train_validation(user)
        train_data = data_split["train_data"]
        train_labels = data_split["train_labels"]
        validation_data = data_split["validation_data"]
        validation_labels = data_split["validation_labels"]


        model = make_model("32-64", img_rows, img_cols, nb_classes)
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy',
                      optimizer=sgd,
                      metrics=['accuracy'])

        model.fit(
            train_data,
            train_labels,
            batch_size=batch_size,
            epochs=nb_epoch,
            shuffle=True, 
            verbose=1)
        y_pred = model.predict_classes(test_data)
        evaluation = model.evaluate(validation_data, validation_labels,
                                    batch_size=batch_size, verbose=0)

        average_loss += evaluation[0]
        average_accuracy += evaluation[1]
        
    return -(average_accuracy/len(users)) # return negative as minimizing

### Callbacks

In [0]:
checkpoint = CheckpointSaver(RESULTS_PATH + "res_gp_checkpoint.pkl")
callbacks_list = [checkpoint]

### Perform Search

In [0]:
res_gp = gp_minimize(objective, space, n_calls=8, n_random_starts=2,
                     random_state=0, callback=callbacks_list)

## Results

## Save results

In [0]:
with open(RESULTS_PATH + "results_dictionary.pkl", 'wb') as results_file:
    pickle.dump(results, results_file)

## Load previous results

In [0]:
with open(RESULTS_PATH + "results_dictionary.pkl", 'rb') as results_file:
    results = pickle.load(results_file)

## Results Visualization

In [0]:
SAVE_GRAPHS = False

### Average Accuracy

In [0]:
accuracies = []
labels = []
for model_name, value in results.items():
    accuracies.append(value[avg_acc])
    labels.append(model_name)
    
plt.bar(range(len(labels)), accuracies)
plt.xticks(range(len(labels)), labels)
plt.xlabel("Model")
plt.ylabel("Average Accuracy")
plt.title("Average Accuracy Comparison from 5-Fold Cross-Validation")
if SAVE_GRAPHS:
    plt.savefig(RESULTS_PATH + "average_accuracy_comparison.pdf", format='pdf')
plt.show()

### Avererage Loss

In [0]:
loss = []
labels = []
for model_name, value in results.items():
    loss.append(value[avg_loss])
    labels.append(model_name)
    
plt.bar(range(len(labels)), loss)
plt.xticks(range(len(labels)), labels)
plt.xlabel("Model")
plt.ylabel("Average Loss")
plt.title("Average Loss Comparison from 5-Fold Cross-Validation")
if SAVE_GRAPHS:
    plt.savefig(RESULTS_PATH + "average_loss_comparison.pdf", format='pdf')
plt.show()